In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests

from RGGI_plant_analysis import RGGI_capacity
from Analyse_PJM import run_historical_RGGI_share,run_future_RGGI_share

In [2]:
historical_rggi_share_pjm,test_df = run_historical_RGGI_share(read_latest=True)

In [3]:
historical_rggi_share_pjm['RGGI_share'] = historical_rggi_share_pjm[1]/historical_rggi_share_pjm[[1,0]].sum(axis=1)

In [4]:
rggi_pjm_rggi_plus_retirements_plus_additions_timeseries,pjm_plus_retirements_plus_additions_timeseries,PJM_plants =  run_future_RGGI_share()

saving down data for 2024-09-01
Operating
Canceled or Postponed
Operating_PR
Planned
Planned_PR
Retired
Retired_PR


c:\GitHub\RGGI_capacity_data\Analyse_PJM.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PJM_plants['PJM_tech'] = PJM_plants['Technology'].map(tech_convert_dict)
c:\GitHub\RGGI_capacity_data\Analyse_PJM.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PJM_plants['Fossil'] = np.where(PJM_plants['PJM_tech'].isin(Fossil_tech),1,0)
c:\GitHub\RGGI_capacity_data\PJM_retirements.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [5]:
test_df = test_df.query('`Balancing Authority Code`=="PJM"').query('report_month==9 and report_year==2024')

In [6]:
test_df.query('PJM_tech=="Coal" and RGGI_state==1')#.groupby(['RGGI_state','PJM_tech'])['Nameplate Capacity (MW)'].sum().unstack('RGGI_state')

,index,Entity ID,Entity Name,Plant ID,Plant Name,Google Map,Bing Map,Plant State,County,Balancing Authority Code,...,Latitude,Longitude,report_month,report_year,Fossil,PlantName_GenID,PJM_tech,dual_fossil,RGGI_state,RGGI_eligible
27453,998,60421,Brandon Shores LLC,602.0,Brandon Shores,Map,Map,MD,Anne Arundel,PJM,...,39.182073,-76.53715,9,2024,1,Brandon Shores 1,Coal,1,1,1
27454,999,60421,Brandon Shores LLC,602.0,Brandon Shores,Map,Map,MD,Anne Arundel,PJM,...,39.182073,-76.53715,9,2024,1,Brandon Shores 2,Coal,1,1,1


In [7]:
PJM_plants.query('PJM_tech=="Coal" and rggi_state==1')

,Entity ID,Entity Name,Plant ID,Plant Name,Google Map,Bing Map,Plant State,County,Balancing Authority Code,Sector,...,Fossil,dual_fossil,PlantName_GenID,year,month,day,retirement_date,PJM_projected_retirement_date,retirement_date_original,rggi_state
998,60421,Brandon Shores LLC,602.0,Brandon Shores,Map,Map,MD,Anne Arundel,PJM,IPP Non-CHP,...,1,1,Brandon Shores 1,2025,6,1,2028-12-31,2028-12-31,2025-06-01,1
999,60421,Brandon Shores LLC,602.0,Brandon Shores,Map,Map,MD,Anne Arundel,PJM,IPP Non-CHP,...,1,1,Brandon Shores 2,2025,6,1,2028-12-31,2028-12-31,2025-06-01,1


In [8]:
PJM_plants.query('`Plant ID`==602')[['PJM_tech','Technology','Nameplate Capacity (MW)','Energy Source Code']] #.groupby(['rggi_state','PJM_tech'])['Nameplate Capacity (MW)'].sum().unstack('rggi_state')

,PJM_tech,Technology,Nameplate Capacity (MW),Energy Source Code
998,Coal,Conventional Steam Coal,685.1,BIT
999,Coal,Conventional Steam Coal,685.1,BIT


In [9]:
PJM_plants#.query('`Plant ID`==593')[['Prime Mover Code','Technology','PJM_tech','Nameplate Capacity (MW)','RGG']]

,Entity ID,Entity Name,Plant ID,Plant Name,Google Map,Bing Map,Plant State,County,Balancing Authority Code,Sector,...,Fossil,dual_fossil,PlantName_GenID,year,month,day,retirement_date,PJM_projected_retirement_date,retirement_date_original,rggi_state
115,5580,"East Kentucky Power Coop, Inc",54.0,J K Smith,Map,Map,KY,Clark,PJM,Electric Utility,...,1,1,J K Smith GT1,2100,12,1,2100-12-01,NaT,2100-12-01,0
116,5580,"East Kentucky Power Coop, Inc",54.0,J K Smith,Map,Map,KY,Clark,PJM,Electric Utility,...,1,1,J K Smith GT10,2100,12,1,2100-12-01,NaT,2100-12-01,0
117,5580,"East Kentucky Power Coop, Inc",54.0,J K Smith,Map,Map,KY,Clark,PJM,Electric Utility,...,1,1,J K Smith GT2,2100,12,1,2100-12-01,NaT,2100-12-01,0
118,5580,"East Kentucky Power Coop, Inc",54.0,J K Smith,Map,Map,KY,Clark,PJM,Electric Utility,...,1,1,J K Smith GT3,2100,12,1,2100-12-01,NaT,2100-12-01,0
119,5580,"East Kentucky Power Coop, Inc",54.0,J K Smith,Map,Map,KY,Clark,PJM,Electric Utility,...,1,1,J K Smith GT4,2100,12,1,2100-12-01,NaT,2100-12-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26352,60520,SoCore Energy LLC,67810.0,NJ Airport Solar Project C,Map,Map,NJ,Essex,PJM,IPP Non-CHP,...,0,1,NJ Airport Solar Project C PV1,2100,12,1,2100-12-01,NaT,2100-12-01,1
26353,60520,SoCore Energy LLC,67811.0,NJ Airport Solar Project D,Map,Map,NJ,Essex,PJM,IPP Non-CHP,...,0,1,NJ Airport Solar Project D PV1,2100,12,1,2100-12-01,NaT,2100-12-01,1
26382,58871,Citizens Enterprises Corporation,67899.0,PNY BESS LLC,Map,Map,PA,Philadelphia,PJM,IPP Non-CHP,...,0,1,PNY BESS LLC GEN5,2100,12,1,2100-12-01,NaT,2100-12-01,0
26396,65677,Dimension Energy LLC,67933.0,Halifax CSG LLC,Map,Map,VA,Halifax,PJM,IPP Non-CHP,...,0,1,Halifax CSG LLC HALIF,2100,12,1,2100-12-01,NaT,2100-12-01,0


In [10]:
rggi_share_pjm = pjm_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('PJM').join(rggi_pjm_rggi_plus_retirements_plus_additions_timeseries.iloc[1:].stack().to_frame('RGGI_PJM'))
rggi_share_pjm = rggi_share_pjm.reset_index()


In [11]:
rggi_share_pjm['RGGI_share'] = rggi_share_pjm['RGGI_PJM']/rggi_share_pjm['PJM']

In [12]:
rggi_share_pjm['report_month'] = rggi_share_pjm.level_0.dt.month
rggi_share_pjm['report_year'] = rggi_share_pjm.level_0.dt.year

In [13]:
pjm_plus_retirements_plus_additions_timeseries

,Coal,Gas,Hydro,Multiple Fuels,Nuclear,Oil,Other,Other Renewables,Solar,Storage,Wind
2024-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-09-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1073.9,12697.7,4697.0,10854.5
2024-10-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1073.9,13779.7,4697.0,10854.5
2024-11-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1076.7,14345.2,4697.0,10854.5
2024-12-01,31049.5,94939.6,3011.7,25764.8,30167.2,5082.5,752.0,1076.7,14838.9,4702.0,11043.5
...,...,...,...,...,...,...,...,...,...,...,...
2030-08-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-09-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-10-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-11-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1


In [14]:
rggi_pjm_rggi_plus_retirements_plus_additions_timeseries.loc['2025-06']

,Coal,Gas,Hydro,Multiple Fuels,Nuclear,Oil,Other,Other Renewables,Solar,Storage,Wind
2025-06-01,1370.2,16470.6,530.8,5856.9,3141.1,1264.7,242.8,0.0,548.1,453.0,234.9


In [15]:
pjm_plus_retirements_plus_additions_timeseries.loc['2025-06']

,Coal,Gas,Hydro,Multiple Fuels,Nuclear,Oil,Other,Other Renewables,Solar,Storage,Wind
2025-06-01,31049.5,94948.1,3014.5,24982.8,30167.2,4769.1,752.0,1076.7,15860.5,4722.0,11098.4


In [16]:
rggi_share_pjm['level_0'] = rggi_share_pjm['level_0']+pd.offsets.MonthEnd(0)

In [17]:
rggi_share_pjm = rggi_share_pjm.rename(columns={'level_1':'PJM_tech',
                                                'level_0':'Date'})

In [18]:
historical_and_forecast = pd.concat([historical_rggi_share_pjm,rggi_share_pjm[['Date','PJM_tech','RGGI_share','report_month','report_year']]],axis=0,join='outer')#.to_csv('RGGI_share_PJM.csv')

In [19]:
historical_and_forecast.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0).to_csv('RGGI_share_PJM.csv')

In [20]:
historical_and_forecast.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0).loc[2024,9]#.query('report_year==2024 and report_month==9')

RGGI_share
Date       PJM_tech                    
2024-09-30 Coal                0.044130
           Gas                 0.173462
           Hydro               0.176246
           Multiple Fuels      0.227322
           Nuclear             0.104123
           Oil                 0.306837
           Other               0.322872
           Other Renewables    0.000000
           Solar               0.020405
           Storage             0.096445
           Wind                0.016583

In [21]:
rggi_share_pjm.query('report_month==11 and report_year==2024')

,Date,PJM_tech,PJM,RGGI_PJM,RGGI_share,report_month,report_year
22,2024-11-30,Coal,31049.5,1370.2,0.044130,11,2024
23,2024-11-30,Gas,94952.1,16470.6,0.173462,11,2024
24,2024-11-30,Hydro,3011.7,530.8,0.176246,11,2024
25,2024-11-30,Multiple Fuels,25764.8,5856.9,0.227322,11,2024
26,2024-11-30,Nuclear,30167.2,3141.1,0.104123,11,2024
27,2024-11-30,Oil,5082.5,1559.5,0.306837,11,2024
28,2024-11-30,Other,752.0,242.8,0.322872,11,2024
29,2024-11-30,Other Renewables,1076.7,0.0,0.000000,11,2024
30,2024-11-30,Solar,14345.2,259.1,0.018062,11,2024
31,2024-11-30,Storage,4697.0,453.0,0.096445,11,2024


In [22]:
historical_and_forecast.query('report_year==2025 and report_month==6')

,report_year,report_month,PJM_tech,0,1,Date,RGGI_share
99,2025,6,Coal,NaN,NaN,2025-06-30,0.044130
100,2025,6,Gas,NaN,NaN,2025-06-30,0.173470
101,2025,6,Hydro,NaN,NaN,2025-06-30,0.176082
102,2025,6,Multiple Fuels,NaN,NaN,2025-06-30,0.234437
103,2025,6,Nuclear,NaN,NaN,2025-06-30,0.104123
104,2025,6,Oil,NaN,NaN,2025-06-30,0.265186
105,2025,6,Other,NaN,NaN,2025-06-30,0.322872
106,2025,6,Other Renewables,NaN,NaN,2025-06-30,0.000000
107,2025,6,Solar,NaN,NaN,2025-06-30,0.034558
108,2025,6,Storage,NaN,NaN,2025-06-30,0.095934


In [23]:
px.line(historical_and_forecast,x='Date',y='RGGI_share',color='PJM_tech')

In [24]:
rggi_pjm_rggi_plus_retirements_plus_additions_timeseries

,Coal,Gas,Hydro,Multiple Fuels,Nuclear,Oil,Other,Other Renewables,Solar,Storage,Wind
2024-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-09-01,1370.2,16470.6,530.8,5856.9,3141.1,1559.5,242.8,0.0,259.1,453.0,180.0
2024-10-01,1370.2,16470.6,530.8,5856.9,3141.1,1559.5,242.8,0.0,259.1,453.0,180.0
2024-11-01,1370.2,16470.6,530.8,5856.9,3141.1,1559.5,242.8,0.0,259.1,453.0,180.0
2024-12-01,1370.2,16470.6,530.8,5856.9,3141.1,1559.5,242.8,0.0,259.1,453.0,180.0
...,...,...,...,...,...,...,...,...,...,...,...
2030-08-01,0.0,16470.6,530.8,5411.4,3141.1,491.0,242.8,0.0,960.0,453.0,1200.9
2030-09-01,0.0,16470.6,530.8,5411.4,3141.1,491.0,242.8,0.0,960.0,453.0,1200.9
2030-10-01,0.0,16470.6,530.8,5411.4,3141.1,491.0,242.8,0.0,960.0,453.0,1200.9
2030-11-01,0.0,16470.6,530.8,5411.4,3141.1,491.0,242.8,0.0,960.0,453.0,1200.9


In [25]:
pjm_plus_retirements_plus_additions_timeseries

,Coal,Gas,Hydro,Multiple Fuels,Nuclear,Oil,Other,Other Renewables,Solar,Storage,Wind
2024-09-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-09-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1073.9,12697.7,4697.0,10854.5
2024-10-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1073.9,13779.7,4697.0,10854.5
2024-11-01,31049.5,94952.1,3011.7,25764.8,30167.2,5082.5,752.0,1076.7,14345.2,4697.0,10854.5
2024-12-01,31049.5,94939.6,3011.7,25764.8,30167.2,5082.5,752.0,1076.7,14838.9,4702.0,11043.5
...,...,...,...,...,...,...,...,...,...,...,...
2030-08-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-09-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-10-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1
2030-11-01,25110.3,96477.0,3198.7,23642.5,30167.2,3995.4,752.0,1076.7,24440.0,5202.0,15712.1


In [26]:
rggi_share_pjm

,Date,PJM_tech,PJM,RGGI_PJM,RGGI_share,report_month,report_year
0,2024-09-30,Coal,31049.5,1370.2,0.044130,9,2024
1,2024-09-30,Gas,94952.1,16470.6,0.173462,9,2024
2,2024-09-30,Hydro,3011.7,530.8,0.176246,9,2024
3,2024-09-30,Multiple Fuels,25764.8,5856.9,0.227322,9,2024
4,2024-09-30,Nuclear,30167.2,3141.1,0.104123,9,2024
...,...,...,...,...,...,...,...
831,2030-12-31,Other,752.0,242.8,0.322872,12,2030
832,2030-12-31,Other Renewables,1076.7,0.0,0.000000,12,2030
833,2030-12-31,Solar,24440.0,960.0,0.039280,12,2030
834,2030-12-31,Storage,5202.0,453.0,0.087082,12,2030


In [27]:

rggi_share_pjm = rggi_share_pjm.groupby(['level_0','PJM_tech'])[['PJM','RGGI_PJM']].sum()
rggi_share_pjm['RGGI_share_PJM'] = rggi_share_pjm['RGGI_PJM']/rggi_share_pjm['PJM']
rggi_share_pjm['RGGI_share_PJM'] = rggi_share_pjm['RGGI_share_PJM'].fillna(0)

KeyError: 'level_0'

In [ ]:
RGGI_coats = pd.read_pickle(fr'C:\Users\Alex Child\OneDrive - Carbon Cap\Carbon markets research\US Markets\RGGI\data\RGGI COATS\Historical_accounts\COATS_report_2024-11-19.pkl')

In [ ]:
RGGI_coats[['Talen' in x for x in RGGI_coats['Account Name']]]

In [ ]:

tech_convert_dict = {'Petroleum Liquids':'Oil',
                    'Onshore Wind Turbine':'Wind',
       'Conventional Hydroelectric':'Hydro', 
       'Conventional Steam Coal':'Coal',
       'Natural Gas Fired Combined Cycle':'Gas', 
       'Natural Gas Steam Turbine':'Gas',
       'Natural Gas Fired Combustion Turbine':'Gas', 
       'Nuclear':'Nuclear',
       'Hydroelectric Pumped Storage':'Storage',
       'Natural Gas Internal Combustion Engine':'Gas', 
       'Batteries':'Storage',
       'Solar Photovoltaic':'Solar', 
       'Geothermal':'Other Renewables', 
       'Wood/Wood Waste Biomass':'Other Renewables',
       'Coal Integrated Gasification Combined Cycle':'Gas',
        'Other Gases':'Gas',
       'Petroleum Coke':'Coal', 
       'Municipal Solid Waste':'Other', 
       'Landfill Gas':'Other Renewables',
       'Natural Gas with Compressed Air Storage':'Gas', 
       'All Other':'Other',
       'Other Waste Biomass':'Other Renewables', 
       'Solar Thermal without Energy Storage':'Solar',
       'Other Natural Gas':'Gas', 
       'Solar Thermal with Energy Storage':'Solar',
       'Flywheels':'Other', 
       'Offshore Wind Turbine':'Wind', 
       'Hydrokinetic':'Hydro',
       'Other Energy Storage':'Storage'}

Fossil_tech = ['Coal','Gas','Oil']

#tCO2/MWh
simple_emissions_factor={'Coal': 1.046,
                           "Gas": 0.467,
                           "Oil": 0.867,
                           'Multiple Fuels':0.467*0.9+0.867*.1}

metric_tons_to_short_tons = 1.1023